In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np 
np.random.seed(0)
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
age_gender = pd.read_csv('data/age_gender_info.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [4]:
test['등록차량수'] = 0

### 결측값 처리

In [5]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [6]:
test.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
등록차량수                             0
dtype: int64

#### 임대보증금 임대료 결측치 처리

In [7]:
display(train.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python'))
display(test.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python'))

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
88,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
89,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
90,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
91,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
101,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.0,D,NaN,NaN,2.0,5.0,428.0,0
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0
81,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,C1729,627,상가,강원도,임대상가,32.46,1,0.0,D,NaN,NaN,0.0,5.0,131.0,0
329,C1729,627,상가,강원도,임대상가,34.86,1,0.0,D,NaN,NaN,0.0,5.0,131.0,0
330,C1729,627,상가,강원도,임대상가,35.76,1,0.0,D,NaN,NaN,0.0,5.0,131.0,0
331,C1729,627,상가,강원도,임대상가,50.08,1,0.0,D,NaN,NaN,0.0,5.0,131.0,0


In [8]:
display(train.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python').공급유형.value_counts())
display(test.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python').공급유형.value_counts())

임대상가    562
공공분양      7
Name: 공급유형, dtype: int64

임대상가    177
영구임대      3
Name: 공급유형, dtype: int64

In [9]:
null_임대_test = test.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python')
null_임대_test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.0,D,NaN,NaN,2.0,5.0,428.0,0
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0
81,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0


In [10]:
display(train.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python').자격유형.value_counts())
display(test.query('임대보증금.isnull() | 임대료.isnull() == True', engine='python').자격유형.value_counts())

D    569
Name: 자격유형, dtype: int64

D    180
Name: 자격유형, dtype: int64

In [11]:
display(train.자격유형.value_counts())
display(test.자격유형.value_counts())

A    1801
D     569
H     155
J     114
C      95
I      49
E      37
K      33
L      33
N      30
B      21
G       9
F       3
M       2
O       1
Name: 자격유형, dtype: int64

A    572
D    180
H     92
J     84
C     34
K     16
L     12
E     10
N     10
I      7
M      2
G      1
Name: 자격유형, dtype: int64

자격 유형 D는 모두 임대료, 보증금이 null 값이다. 주로 임대상가에 해당 -> 임대보증금, 임대료 산정 불가

In [12]:
train[['임대보증금','임대료']] = train[['임대보증금', '임대료']].fillna("0").replace("-", "0").astype(int)
test[['임대보증금','임대료']] = test[['임대보증금', '임대료']].fillna("0").replace("-", "0").astype(int)

In [13]:
display(train[['임대보증금','임대료']].isnull().sum())
display(test[['임대보증금','임대료']].isnull().sum())

임대보증금    0
임대료      0
dtype: int64

임대보증금    0
임대료      0
dtype: int64

#### 자격유형 결측값 대체

In [14]:
display(train.query('자격유형.isnull() == True', engine='python'))
display(test.query('자격유형.isnull() == True', engine='python'))

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0,0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0,0


In [15]:
display(train.query('공급유형 == "국민임대" & 임대건물구분 == "아파트"')['자격유형'].value_counts())
display(train.query('공급유형 == "영구임대" & 임대건물구분 == "아파트"')['자격유형'].value_counts())

display(test.query('공급유형 == "국민임대" & 임대건물구분 == "아파트"')['자격유형'].value_counts())
display(test.query('공급유형 == "영구임대" & 임대건물구분 == "아파트"')['자격유형'].value_counts())

A    1539
H     155
E      34
B      21
G       9
Name: 자격유형, dtype: int64

C    95
I    49
E     3
F     3
A     2
Name: 자격유형, dtype: int64

A    518
H     92
E     10
G      1
Name: 자격유형, dtype: int64

C    34
I     7
D     3
Name: 자격유형, dtype: int64

국민임대&아파트인 경우 -> 유형'A'가 가장 많았음  
영구임대&아파트인 경우 -> 유형'C'가 가장 많았음

In [16]:
test['자격유형'].iloc[196] = "A"
test['자격유형'].iloc[258] = "A"

In [17]:
display(train[['임대보증금','임대료']].isnull().sum())
display(test[['임대보증금','임대료']].isnull().sum())

임대보증금    0
임대료      0
dtype: int64

임대보증금    0
임대료      0
dtype: int64

In [18]:
train.자격유형.value_counts()

A    1801
D     569
H     155
J     114
C      95
I      49
E      37
K      33
L      33
N      30
B      21
G       9
F       3
M       2
O       1
Name: 자격유형, dtype: int64

### 지하철역 수, 버스정류장 수 결측값 대체

편의를 위한 열이름 변경

In [19]:
train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철'},inplace =  True)
train.rename(columns={'도보 10분거리 내 버스정류장 수':'버스'}, inplace = True)
test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철'},inplace =  True)
test.rename(columns={'도보 10분거리 내 버스정류장 수':'버스'}, inplace = True)

#### 지하철 결측값 대체

In [20]:
display(train.query('지하철.isnull() == True', engine='python').head())
display(test.query('지하철.isnull() == True', engine='python').head())

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
94,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,A,17460000,122210,NaN,3.0,527.0,359.0
95,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,A,17460000,122210,NaN,3.0,527.0,359.0
96,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,A,19954000,130940,NaN,3.0,527.0,359.0
97,C1312,518,아파트,충청남도,국민임대,46.90,143,12.0,A,28687000,149660,NaN,3.0,527.0,359.0
98,C1874,619,아파트,충청남도,영구임대,26.37,294,2.0,C,3141000,69900,NaN,2.0,97.0,62.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
68,C1472,548,아파트,충청남도,국민임대,36.62,116,2.0,A,8738000,87380,NaN,1.0,442.0,0
69,C1472,548,아파트,충청남도,국민임대,39.51,56,2.0,A,9985000,99850,NaN,1.0,442.0,0
70,C1472,548,아파트,충청남도,국민임대,46.79,56,2.0,A,13731000,131070,NaN,1.0,442.0,0
71,C1472,548,아파트,충청남도,국민임대,46.90,86,2.0,A,13731000,131070,NaN,1.0,442.0,0
72,C1472,548,아파트,충청남도,국민임대,51.93,234,2.0,A,18726000,143550,NaN,1.0,442.0,0


In [21]:
display(train.지하철.value_counts())
display(test.지하철.value_counts())

0.0    2302
1.0     400
2.0      33
3.0       6
Name: 지하철, dtype: int64

0.0    881
1.0     64
2.0     35
Name: 지하철, dtype: int64

지하철 수 null값은 최빈값인 0으로 대체

In [22]:
train['지하철'] = train['지하철'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

In [23]:
display(train[['지하철']].isnull().sum())
display(test[['지하철']].isnull().sum())

지하철    0
dtype: int64

지하철    0
dtype: int64

#### 버스 결측값 대체

In [24]:
display(train.query('버스.isnull() == True', engine='python').head())
display(test.query('버스.isnull() == True', engine='python').head())

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2315,C1649,575,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,0.0,NaN,1066.0,855.0
2316,C1649,575,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0
2317,C1649,575,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0
2318,C1649,575,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수


In [25]:
a = train.버스.value_counts().index[0]
train['버스'] = train['버스'].fillna(a)

In [26]:
display(train[['버스']].isnull().sum())

버스    0
dtype: int64

## 코드별 집계

In [27]:
# 단지 코드별 같은 값이 나오는 열
unique_cols = ['총세대수','지역','공가수','지하철','버스','단지내주차면수','등록차량수']

train_code_agg = train.set_index('단지코드')[unique_cols].drop_duplicates().reset_index()
test_code_agg = test.set_index('단지코드')[unique_cols].drop_duplicates().reset_index()

### 코드별 집계 - 다른열 (임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료)

In [28]:
# count를 이용한 피벗테이블을 만들기 위해 임시로 forcount 변수를 만들어 준다
train['forcount'] = 1
test['forcount'] = 1

#### 임대건물구분

In [29]:
data_list = [train,test]

In [30]:
def rent_pivot(data_list):
    frame_list = []
    for data in data_list:
        pivot = pd.pivot_table(data.drop_duplicates(['단지코드','임대건물구분']), index = '단지코드',
                     columns = '임대건물구분',
                      values = 'forcount',
                      aggfunc = 'count',
                      fill_value = 0)
        pivot.columns.name = None
        pivot = pivot.rename(columns = {cols: '임대건물_'+cols for cols in pivot.columns})
        pivot = pivot.reset_index()
        pivot = pd.DataFrame(pivot)
        frame_list.append(pivot)
        
    return frame_list

(train_rent_pivot,test_rent_pivot) = rent_pivot(data_list)

In [31]:
train_code_agg = pd.merge(train_code_agg,train_rent_pivot)
test_code_agg = pd.merge(test_code_agg,test_rent_pivot)

#### 자격유형

공급유형과 자격유형이 어느정도 관련이 있을 것.

In [32]:
train.groupby(['공급유형','자격유형'])['자격유형'].count()

공급유형       자격유형
공공분양       D          7
공공임대(10년)  A        205
공공임대(50년)  A         31
공공임대(5년)   A          3
공공임대(분납)   A         12
국민임대       A       1539
           B         21
           E         34
           G          9
           H        155
영구임대       A          2
           C         95
           E          3
           F          3
           I         49
임대상가       D        562
장기전세       A          9
행복주택       J        114
           K         33
           L         33
           M          2
           N         30
           O          1
Name: 자격유형, dtype: int64

In [33]:
test.groupby(['공급유형','자격유형'])['자격유형'].count()

공급유형       자격유형
공공임대(10년)  A        35
공공임대(50년)  A        13
공공임대(분납)   A         6
국민임대       A       519
           E        10
           G         1
           H        92
영구임대       A         1
           C        34
           D         3
           I         7
임대상가       D       177
행복주택       J        84
           K        16
           L        12
           M         2
           N        10
Name: 자격유형, dtype: int64

자격유형은 비식별화 되어있기 때문에 정확하게는 알 수 없지만 추측은 가능. 
공급유형에 따른 자격유형 그룹화를 진행.  
* A 유형은 여러 곳에서 분포하며, 특히 국민임대에 집중되어 있는 것을 보면 '무주택자'로 보임.  
* 영구임대의 공급대상은 기초생활수급자, 독거노인, 국가유공자 등 사회적 취약계층으로 이루어져 있음.  
* 행복주택의 공급대상은 대학생, 사회 초년생, 신혼부부 등으로 이루어져 있음.  
(특히 행복주택의 경우 해당되는 자격유형이 J,K,L,M,N,O로 다른 공급유형과 겹치지 않음.)  
A 와 D 유형의 경우 여러 공급유형에 분포되어있기에 그룹핑 X

In [34]:
for data in data_list:
    for i in tqdm(range(data.shape[0])):
        if data['자격유형'].iloc[i] in ['B','E','G','H']:
            data['자격유형'].iloc[i] = '국민임대/장기전세공급대상'
        elif data['자격유형'].iloc[i] in ['C','F','I']:
            data['자격유형'].iloc[i] = '영구임대공급대상'
        elif data['자격유형'].iloc[i] in ['J','K','L','M','N','O']:
            data['자격유형'].iloc[i] = '행복주택공급대상'
        else:
            pass

100%|██████████| 1022/1022 [00:00<00:00, 2800.82it/s]


In [35]:
def qualification_pivot(data_list):
    frame_list = []
    for data in data_list:
        pivot = pd.pivot_table(data.drop_duplicates(['단지코드','자격유형']), index = '단지코드',
                     columns = '자격유형',
                      values = 'forcount',
                      aggfunc = 'count',
                      fill_value = 0)
        pivot.columns.name = None
        pivot = pivot.rename(columns = {cols: '자격유형_'+cols for cols in pivot.columns})
        pivot = pivot.reset_index()
        pivot = pd.DataFrame(pivot)
        frame_list.append(pivot)
        
    return frame_list

(train_qualification_pivot,test_qualification_pivot) = qualification_pivot(data_list)

In [36]:
train_code_agg = pd.merge(train_code_agg,train_qualification_pivot)
test_code_agg = pd.merge(test_code_agg,test_qualification_pivot)

In [37]:
train.자격유형.value_counts()

A                1801
D                 569
국민임대/장기전세공급대상     222
행복주택공급대상          213
영구임대공급대상          147
Name: 자격유형, dtype: int64

In [38]:
test.자격유형.value_counts()

A                574
D                180
행복주택공급대상         124
국민임대/장기전세공급대상    103
영구임대공급대상          41
Name: 자격유형, dtype: int64

In [39]:
train_code_agg

,단지코드,총세대수,지역,공가수,지하철,버스,단지내주차면수,등록차량수,임대건물_상가,임대건물_아파트,자격유형_A,자격유형_D,자격유형_국민임대/장기전세공급대상,자격유형_영구임대공급대상,자격유형_행복주택공급대상
0,C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0,0,1,1,0,0,0,0
1,C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0,0,1,1,0,0,0,0
2,C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0,0,1,1,0,0,0,0
3,C1945,755,경기도,6.0,1.0,3.0,734.0,730.0,0,1,0,0,1,0,0
4,C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90,제주특별자치도,7.0,0.0,3.0,66.0,57.0,0,1,0,0,0,0,1
419,C2035,492,강원도,24.0,0.0,1.0,521.0,246.0,0,1,1,0,0,0,0
420,C2020,40,부산광역시,7.0,1.0,2.0,25.0,19.0,0,1,0,0,0,0,1
421,C2437,90,충청북도,12.0,0.0,1.0,30.0,16.0,0,1,0,0,0,1,0


## age_gender 데이터 활용

In [40]:
age_gender['0~10대(여자)'] = age_gender['10대미만(여자)'] + age_gender['10대(여자)']
age_gender['0~10대(남자)'] = age_gender['10대미만(남자)'] + age_gender['10대(남자)']
age_gender['20~30대(여자)'] = age_gender['20대(여자)'] + age_gender['30대(여자)']
age_gender['20~30대(남자)'] = age_gender['20대(남자)'] + age_gender['30대(남자)']
age_gender['40~50대(여자)'] = age_gender['40대(여자)'] + age_gender['50대(여자)']
age_gender['40~50대(남자)'] = age_gender['40대(남자)'] + age_gender['50대(남자)']
age_gender['60~70대(여자)'] = age_gender['60대(여자)'] + age_gender['70대(여자)']
age_gender['60~70대(남자)'] = age_gender['60대(남자)'] + age_gender['70대(남자)']
age_gender['80~100대'] = age_gender['80대(여자)'] + age_gender['90대(여자)'] + age_gender['100대(여자)'] + age_gender['80대(남자)'] + age_gender['90대(남자)'] + age_gender['100대(남자)']
AG = age_gender.iloc[:, 23:32]
AG['지역'] = age_gender['지역']

In [41]:
AG.iloc[:,0:9] = AG.iloc[:,0:9].rank()

In [42]:
train_code_agg = pd.merge(train_code_agg, AG)
test_code_agg = pd.merge(test_code_agg, AG)

#### 세대당주차면수

In [43]:
train_code_agg['세대당주차면수'] = train_code_agg['단지내주차면수'] / train_code_agg['총세대수']
test_code_agg['세대당주차면수'] = test_code_agg['단지내주차면수'] / test_code_agg['총세대수']

#### 임대면적

In [44]:
train['임대면적'] = train.전용면적 * train.전용면적별세대수
test['임대면적'] = test.전용면적 * test.전용면적별세대수

#### 단지코드별 총임대면적

In [45]:
frame_list = []
for data in data_list:
    frame = data.groupby('단지코드')['임대면적'].agg('sum').reset_index()
    frame.columns = ['단지코드','총임대면적']
    frame_list.append(frame)

(train_total_rent,test_total_rent) = frame_list

train_code_agg = pd.merge(train_code_agg, train_total_rent)
train_code_agg.iloc[:,-1] = train_code_agg.iloc[:,-1].fillna(0)

test_code_agg = pd.merge(test_code_agg, test_total_rent)
test_code_agg.iloc[:,-1] = test_code_agg.iloc[:,-1].fillna(0)    

In [46]:
train_code_agg['세대당평균전용면적'] = train_code_agg.총임대면적 / train_code_agg.총세대수
test_code_agg['세대당평균전용면적'] = train_code_agg.총임대면적 / train_code_agg.총세대수

#### 단지코드별 임대료, 임대보증금 평균

In [47]:
def 임대료평균(a):
    임평 = a.groupby('단지코드')['임대료'].agg('mean').reset_index()
    임평.columns = ['단지코드','임대료평균']
    return 임평
def 임대보증금평균(a):
    임보평 = a.groupby('단지코드')['임대보증금'].agg('mean').reset_index()
    임보평.columns = ['단지코드','임대보증금평균']
    return 임보평

train_code_agg = pd.merge(train_code_agg, 임대료평균(train), on = '단지코드')
test_code_agg = pd.merge(test_code_agg, 임대료평균(test), on = '단지코드')

train_code_agg = pd.merge(train_code_agg, 임대보증금평균(train), on = '단지코드')
test_code_agg = pd.merge(test_code_agg, 임대보증금평균(test), on = '단지코드')

#### 공가수 비율 

In [48]:
train_code_agg['공가수비율'] = train_code_agg.공가수 / train_code_agg.총세대수
test_code_agg['공가수비율'] = test_code_agg.공가수 / test_code_agg.총세대수

In [49]:
train_공가수비율_scale = list(train_code_agg.공가수비율.describe().iloc[3:8].values)

In [50]:
def 범주화(x):
    if train_공가수비율_scale[0] <= x < train_공가수비율_scale[1]  :
        return 1
    elif train_공가수비율_scale[1] <= x < train_공가수비율_scale[2]:
        return 2
    elif train_공가수비율_scale[2] <= x < train_공가수비율_scale[3]:
        return 3
    elif train_공가수비율_scale[3] <= x:
        return 4

train_code_agg['공가수비율분류'] = train_code_agg.공가수비율.apply(범주화)
test_code_agg['공가수비율분류'] = test_code_agg.공가수비율.apply(범주화)

#### 임대료, 임대보증금 범주화

In [51]:
train_임대료_scale = list(train_code_agg.임대료평균.describe().iloc[3:8].values)
train_임대보증금_scale = list(train_code_agg.임대보증금평균.describe().iloc[3:8].values)

In [52]:
def 범주화(x):
    if train_임대료_scale[0] <= x < train_임대료_scale[1]  :
        return 1
    elif train_임대료_scale[1] <= x < train_임대료_scale[2]:
        return 2
    elif train_임대료_scale[2] <= x < train_임대료_scale[3]:
        return 3
    elif train_임대료_scale[3] <= x:
        return 4

train_code_agg['임대료분류'] = train_code_agg.임대료평균.apply(범주화)
test_code_agg['임대료분류'] = test_code_agg.임대료평균.apply(범주화)

In [53]:
def 범주화(x):
    if train_임대보증금_scale[0] <= x < train_임대보증금_scale[1]  :
        return 1
    elif train_임대보증금_scale[1] <= x < train_임대보증금_scale[2]:
        return 2
    elif train_임대보증금_scale[2] <= x < train_임대보증금_scale[3]:
        return 3
    elif train_임대보증금_scale[3] <= x:
        return 4

train_code_agg['임대보증금분류'] = train_code_agg.임대보증금평균.apply(범주화)
test_code_agg['임대보증금분류'] = test_code_agg.임대보증금평균.apply(범주화)

In [54]:
train_세대당주차면수_scale = list(train_code_agg.세대당주차면수.describe().iloc[3:8].values)

In [55]:
def 범주화(x):
    if train_세대당주차면수_scale[0] <= x < train_세대당주차면수_scale[1]  :
        return 1
    elif train_세대당주차면수_scale[1] <= x < train_세대당주차면수_scale[2]:
        return 2
    elif train_세대당주차면수_scale[2] <= x < train_세대당주차면수_scale[3]:
        return 3
    elif train_세대당주차면수_scale[3] <= x:
        return 4

train_code_agg['세대당주차면수분류'] = train_code_agg.세대당주차면수.apply(범주화)
test_code_agg['세대당주차면수분류'] = test_code_agg.세대당주차면수.apply(범주화)

In [56]:
train_총임대면적_scale = list(train_code_agg.총임대면적.describe().iloc[3:8].values)

In [57]:
def 범주화(x):
    if train_총임대면적_scale[0] <= x < train_총임대면적_scale[1]  :
        return 1
    elif train_총임대면적_scale[1] <= x < train_총임대면적_scale[2]:
        return 2
    elif train_총임대면적_scale[2] <= x < train_총임대면적_scale[3]:
        return 3
    elif train_총임대면적_scale[3] <= x:
        return 4

train_code_agg['총임대면적분류'] = train_code_agg.총임대면적.apply(범주화)
test_code_agg['총임대면적분류'] = test_code_agg.총임대면적.apply(범주화)

train_공가수_scale = list(train_code_agg.공가수.describe().iloc[3:8].values)

def 범주화(x):
    if train_공가수_scale[0] <= x < train_공가수_scale[1]  :
        return 1
    elif train_공가수_scale[1] <= x < train_공가수_scale[2]:
        return 2
    elif train_공가수_scale[2] <= x < train_공가수_scale[3]:
        return 3
    elif train_공가수_scale[3] <= x:
        return 4

train_code_agg['공가수분류'] = train_code_agg.공가수.apply(범주화)
test_code_agg['공가수분류'] = test_code_agg.공가수.apply(범주화)

In [58]:
train_세대당평균전용면적_scale = list(train_code_agg.세대당평균전용면적.describe().iloc[3:8].values)

In [59]:
def 범주화(x):
    if train_세대당평균전용면적_scale[0] <= x < train_세대당평균전용면적_scale[1]  :
        return 1
    elif train_세대당평균전용면적_scale[1] <= x < train_세대당평균전용면적_scale[2]:
        return 2
    elif train_세대당평균전용면적_scale[2] <= x < train_세대당평균전용면적_scale[3]:
        return 3
    elif train_세대당평균전용면적_scale[3] <= x:
        return 4

train_code_agg['세대당평균전용면적분류'] = train_code_agg.세대당평균전용면적.apply(범주화)
test_code_agg['세대당평균전용면적분류'] = test_code_agg.세대당평균전용면적.apply(범주화)

### mean median std

In [60]:
def with_not_unique_column(code_agg, not_unique_column, handle_things, func_list):
    frame_list = []
    for func in func_list:
        for handle_thing in handle_things:
            grouped_frame = code_agg.groupby(not_unique_column)[handle_thing].agg(func)
            grouped_frame = grouped_frame.reset_index()
            grouped_frame.columns = [not_unique_column, not_unique_column+'_'+ handle_thing +'_'+func]
            frame_list.append(grouped_frame)
    return frame_list

In [61]:
train_geo = with_not_unique_column(train_code_agg, '지역',['단지내주차면수', 
                                                         '총세대수','세대당주차면수'],
                                   ['std'])

def merge_m3(code_agg, total_frame_list):
    for frame_list in total_frame_list:
        for frame in frame_list:
            code_agg = pd.merge(code_agg,frame)
    return code_agg

train_total = [train_geo]

train_code_agg = merge_m3(train_code_agg,train_total)
test_code_agg = merge_m3(test_code_agg,train_total)

**열 삭제**

In [62]:
train_code_agg = train_code_agg.drop(['임대료평균', '임대보증금평균','80~100대' , '버스',  '0~10대(남자)', '자격유형_행복주택공급대상',
                                '임대건물_아파트','0~10대(여자)','60~70대(남자)','세대당주차면수'
                                ],
                           axis='columns')

In [63]:
test_code_agg = test_code_agg.drop(['임대료평균', '임대보증금평균','80~100대',   '버스', '0~10대(남자)', '자격유형_행복주택공급대상',
                                '임대건물_아파트','0~10대(여자)','60~70대(남자)','세대당주차면수'
                                ], 
                           axis='columns')

In [64]:
train_code_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 422
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   단지코드                423 non-null    object 
 1   총세대수                423 non-null    int64  
 2   지역                  423 non-null    object 
 3   공가수                 423 non-null    float64
 4   지하철                 423 non-null    float64
 5   단지내주차면수             423 non-null    float64
 6   등록차량수               423 non-null    float64
 7   임대건물_상가             423 non-null    int64  
 8   자격유형_A              423 non-null    int64  
 9   자격유형_D              423 non-null    int64  
 10  자격유형_국민임대/장기전세공급대상  423 non-null    int64  
 11  자격유형_영구임대공급대상       423 non-null    int64  
 12  20~30대(여자)          423 non-null    float64
 13  20~30대(남자)          423 non-null    float64
 14  40~50대(여자)          423 non-null    float64
 15  40~50대(남자)          423 non-null    float64
 16  60~70대(여

In [65]:
test_code_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   단지코드                150 non-null    object 
 1   총세대수                150 non-null    int64  
 2   지역                  150 non-null    object 
 3   공가수                 150 non-null    float64
 4   지하철                 150 non-null    float64
 5   단지내주차면수             150 non-null    float64
 6   등록차량수               150 non-null    int64  
 7   임대건물_상가             150 non-null    int64  
 8   자격유형_A              150 non-null    int64  
 9   자격유형_D              150 non-null    int64  
 10  자격유형_국민임대/장기전세공급대상  150 non-null    int64  
 11  자격유형_영구임대공급대상       150 non-null    int64  
 12  20~30대(여자)          150 non-null    float64
 13  20~30대(남자)          150 non-null    float64
 14  40~50대(여자)          150 non-null    float64
 15  40~50대(남자)          150 non-null    float64
 16  60~70대(여

In [66]:
from sklearn.preprocessing import StandardScaler

In [67]:
train_code_agg.columns

Index(['단지코드', '총세대수', '지역', '공가수', '지하철', '단지내주차면수', '등록차량수', '임대건물_상가',
       '자격유형_A', '자격유형_D', '자격유형_국민임대/장기전세공급대상', '자격유형_영구임대공급대상', '20~30대(여자)',
       '20~30대(남자)', '40~50대(여자)', '40~50대(남자)', '60~70대(여자)', '총임대면적',
       '세대당평균전용면적', '공가수비율', '공가수비율분류', '임대료분류', '임대보증금분류', '세대당주차면수분류',
       '총임대면적분류', '세대당평균전용면적분류', '지역_단지내주차면수_std', '지역_총세대수_std',
       '지역_세대당주차면수_std'],
      dtype='object')

**스케일링**

In [68]:
scailing_features = ['총세대수','총임대면적','단지내주차면수','지하철']
scaler = StandardScaler()
train_code_agg.loc[:, scailing_features] = scaler.fit_transform(train_code_agg[scailing_features])
test_code_agg.loc[:, scailing_features] = scaler.transform(test_code_agg[scailing_features])

#### 오류 단지코드 삭제

In [69]:
train_code_agg = train_code_agg.set_index('단지코드')
a =  ['C2085','C1397','C2431', 'C1649','C1036','C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
train_code_agg = train_code_agg.drop(a, axis=0)
train_code_agg = train_code_agg.reset_index()

### 모델링

In [70]:
import sklearn
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from catboost import CatBoostRegressor,Pool

In [71]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
catcol = ['지역']
for i in catcol:
    le = LabelEncoder()
    le.fit(train_code_agg[i])
    train_code_agg[i] = le.transform(train_code_agg[i])
    test_code_agg[i] = le.transform(test_code_agg[i])

# 하이퍼 파라미터 튜닝

In [72]:
#'''
target = train_code_agg['등록차량수']
train_data = train_code_agg.drop(['단지코드','등록차량수'], axis=1)

test_data = pd.merge(sample,test_code_agg, left_on = 'code', right_on='단지코드')
test_data = test_data.drop(['code','num','단지코드','등록차량수'], axis=1)
#'''

In [73]:
def objective(trial: Trial):
    params_cat = {
        "random_state": 0,
        "learning_rate": 0.05,
        "n_estimators": 500,
        "verbose" : 0,
        "objective" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.1, 0.5),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 50),
        "max_bin": trial.suggest_int("max_bin", 100, 500),
    }
    
    X_tr, X_val, y_tr, y_val = train_test_split(train_data , target, test_size=0.2)

    model = CatBoostRegressor(**params_cat)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_tr, y_tr), (X_val, y_val)],
        early_stopping_rounds=10,
        verbose=False,
    )

    cat_pred = model.predict(X_val)
    log_score = mean_absolute_error(y_val, cat_pred)
    
    return log_score

In [74]:
sampler = TPESampler(seed = 0)
study = optuna.create_study(
    study_name="cat_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials = 100)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-30 16:57:45,270] A new study created in memory with name: cat_opt
[I 2021-07-30 16:57:46,174] Trial 0 finished with value: 109.91216254690275 and parameters: {'max_depth': 7, 'colsample_bylevel': 0.943037873274484, 'subsample': 0.3411053504286575, 'min_child_samples': 29, 'max_bin': 269}. Best is trial 0 with value: 109.91216254690275.
[I 2021-07-30 16:57:47,062] Trial 1 finished with value: 107.16787964315877 and parameters: {'max_depth': 8, 'colsample_bylevel': 0.8875174422525385, 'subsample': 0.4567092003128319, 'min_child_samples': 49, 'max_bin': 253}. Best is trial 1 with value: 107.16787964315877.
[I 2021-07-30 16:57:48,171] Trial 2 finished with value: 125.25156599156145 and parameters: {'max_depth': 9, 'colsample_bylevel': 0.9057789839505809, 'subsample': 0.32721782443757297, 'min_child_samples': 47, 'max_bin': 128}. Best is trial 1 with value: 107.16787964315877.
[I 2021-07-30 16:57:48,514] Trial 3 finished with value: 90.0050138535686 and parameters: {'max_depth': 

[I 2021-07-30 16:57:59,952] Trial 29 finished with value: 129.20441700189664 and parameters: {'max_depth': 7, 'colsample_bylevel': 0.8363450158903151, 'subsample': 0.49760513380912563, 'min_child_samples': 31, 'max_bin': 500}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:00,239] Trial 30 finished with value: 98.63033024981279 and parameters: {'max_depth': 5, 'colsample_bylevel': 0.8101300807842005, 'subsample': 0.35730192911283276, 'min_child_samples': 14, 'max_bin': 404}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:00,412] Trial 31 finished with value: 111.48212675605133 and parameters: {'max_depth': 5, 'colsample_bylevel': 0.8105524401375293, 'subsample': 0.3452204100833708, 'min_child_samples': 16, 'max_bin': 406}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:00,642] Trial 32 finished with value: 93.4442569551919 and parameters: {'max_depth': 5, 'colsample_bylevel': 0.8012350849521522, 'subsample': 0.4224900732291

[I 2021-07-30 16:58:13,863] Trial 58 finished with value: 136.8347044004928 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.8306174918206359, 'subsample': 0.2228180782789627, 'min_child_samples': 9, 'max_bin': 423}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:14,194] Trial 59 finished with value: 131.56704065673065 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.8044887083624503, 'subsample': 0.15978667613458547, 'min_child_samples': 4, 'max_bin': 171}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:14,532] Trial 60 finished with value: 109.61578007125222 and parameters: {'max_depth': 5, 'colsample_bylevel': 0.8211225849408115, 'subsample': 0.18177820401408368, 'min_child_samples': 12, 'max_bin': 384}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:15,247] Trial 61 finished with value: 120.90000408648869 and parameters: {'max_depth': 7, 'colsample_bylevel': 0.8198344722218834, 'subsample': 0.2093314339958

[I 2021-07-30 16:58:26,143] Trial 87 finished with value: 129.63707068705725 and parameters: {'max_depth': 4, 'colsample_bylevel': 0.8536129954093814, 'subsample': 0.1986240068662856, 'min_child_samples': 9, 'max_bin': 443}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:26,335] Trial 88 finished with value: 92.92313028761286 and parameters: {'max_depth': 5, 'colsample_bylevel': 0.9124483841185722, 'subsample': 0.42538335603545513, 'min_child_samples': 21, 'max_bin': 352}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:26,504] Trial 89 finished with value: 118.41555169091718 and parameters: {'max_depth': 4, 'colsample_bylevel': 0.9019467788812982, 'subsample': 0.46691381930259185, 'min_child_samples': 22, 'max_bin': 281}. Best is trial 12 with value: 85.70632849124807.
[I 2021-07-30 16:58:27,047] Trial 90 finished with value: 130.22315876098872 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.9104923493048078, 'subsample': 0.423821965106

Best Score: 85.70632849124807
Best trial: {'max_depth': 5, 'colsample_bylevel': 0.8083627889054776, 'subsample': 0.22482679942497147, 'min_child_samples': 36, 'max_bin': 421}


In [75]:

cat_p = study.best_trial.params
cat = CatBoostRegressor(**cat_p)

y_cat = pd.cut(target, 10, labels=range(10))
skf = StratifiedKFold(5)

preds = []
for tr_id, val_id in skf.split(train_data, y_cat) : 
    X_tr = train_data.iloc[tr_id]
    y_tr = target.iloc[tr_id]
    
    cat.fit(X_tr, y_tr, verbose = 0)
    
    pred = cat.predict(test_data)
    preds.append(pred)
cat_pred = np.mean(preds, axis = 0)

sample['num'] = cat_pred
sample


,code,num
0,C1072,797.712009
1,C1128,1307.710787
2,C1456,524.475828
3,C1840,469.174274
4,C1332,1241.769862
...,...,...
145,C2456,292.673642
146,C1266,558.471504
147,C2152,181.120384
148,C1267,301.672292


In [76]:
#sample.to_csv('catboost_0729_변수수정_2번.csv', index=False)

In [ ]:
good = pd.read_csv('submission_catb_baseline_미안합니다여러분_72_87.csv')

In [ ]:
np.mean(abs(good['num']-sample['num']))